# Import the Necessary Libraries

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
import shutil

### Step 1: Build U-Net Model
In this step, we will define the architecture of the U-Net model. The U-Net consists of an encoder, bottleneck, and decoder. We will use convolutional layers to extract features, max-pooling layers to downsample, and transpose convolutions for upsampling. Finally, the model will output a binary mask.


In [2]:
def build_unet(input_shape=(256, 256, 3)):
    inputs = Input(input_shape)
    
    # Encoder
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    p1 = MaxPooling2D((2, 2))(c1)
    
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    p2 = MaxPooling2D((2, 2))(c2)
    
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    p3 = MaxPooling2D((2, 2))(c3)
    
    # Bottleneck
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    
    # Decoder
    u1 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c4)
    u1 = concatenate([u1, c3])
    c5 = Conv2D(256, (3, 3), activation='relu', padding='same')(u1)
    
    u2 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u2 = concatenate([u2, c2])
    c6 = Conv2D(128, (3, 3), activation='relu', padding='same')(u2)
    
    u3 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u3 = concatenate([u3, c1])
    c7 = Conv2D(64, (3, 3), activation='relu', padding='same')(u3)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c7)
    
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

### Step 2: Load Images and Masks
In this step, we load the training and testing images from the specified directories. The images and their corresponding binary masks are resized to a uniform size (256x256). The pixel values are normalized to a range between 0 and 1.


In [3]:
image_dir = '/kaggle/input/semantic-segmentation-of-underwater-imagery-suim/train_val/images/'
mask_dir = '/kaggle/input/semantic-segmentation-of-underwater-imagery-suim/train_val/masks/'

images = []
masks = []

for filename in os.listdir(image_dir):
    if filename.endswith('.jpg'):
        image = cv2.imread(os.path.join(image_dir, filename))
        image = cv2.resize(image, (256, 256)) / 255.0
        mask = cv2.imread(os.path.join(mask_dir, filename.replace('.jpg', '.bmp')), 0)
        mask = cv2.resize(mask, (256, 256)) / 255.0
        
        images.append(image)
        masks.append(mask)

# Convert to NumPy Arrays
X = np.array(images)
Y = np.expand_dims(np.array(masks), axis=-1)

### Step 3: Split Data and Train
In this step, we split the dataset into training and testing sets (90% training, 10% testing). Then, we train the U-Net model on the training data for 10 epochs. The model is compiled with the Adam optimizer and binary cross-entropy loss.


In [4]:
unet_model = build_unet()
unet_model.fit(X, Y, epochs=10, batch_size=16, verbose=1)

Epoch 1/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 41s 229ms/step - accuracy: 0.3365 - loss: 0.6560
Epoch 2/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 16s 162ms/step - accuracy: 0.3439 - loss: 0.6457
Epoch 3/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 16s 162ms/step - accuracy: 0.3336 - loss: 0.6416
Epoch 4/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 16s 162ms/step - accuracy: 0.3266 - loss: 0.6429
Epoch 5/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 16s 162ms/step - accuracy: 0.3354 - loss: 0.6371
Epoch 6/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 16s 162ms/step - accuracy: 0.3306 - loss: 0.6397
Epoch 7/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 16s 162ms/step - accuracy: 0.3276 - loss: 0.6444
Epoch 8/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 16s 162ms/step - accuracy: 0.3441 - loss: 0.6321
Epoch 9/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 16s 162ms/step - accuracy: 0.3379 - loss: 0.6320
Epoch 10/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 16s 162ms/step - accuracy: 0.3581 - loss: 0.6262


### Step 4: Save the Model
After training the model, we save it to the `/kaggle/working/` directory. This allows us to use the trained model for inference on the test data.


In [5]:
unet_model.save('/kaggle/working/best_model.keras')
print("✅ Model trained and saved as best_model.keras.")

✅ Model trained and saved as best_model.keras.


### Load Test Images and Masks
In this step, we load the test images and their corresponding binary masks from the specified directories. The images are resized to the required shape (256x256) and normalized. Similarly, the masks are resized and normalized as well.


In [8]:
test_image_dir = '/kaggle/input/semantic-segmentation-of-underwater-imagery-suim/TEST/images'
test_mask_dir = '/kaggle/input/semantic-segmentation-of-underwater-imagery-suim/TEST/masks'

test_images = []
test_masks = []

for filename in os.listdir(test_image_dir):
    if filename.endswith('.jpg'):
        image = cv2.imread(os.path.join(test_image_dir, filename))
        image = cv2.resize(image, (256, 256)) / 255.0
        mask = cv2.imread(os.path.join(test_mask_dir, filename.replace('.jpg', '.bmp')), 0)
        mask = cv2.resize(mask, (256, 256)) / 255.0
        
        test_images.append(image)
        test_masks.append(mask)

### Step 5: Predict and Save Results
In this step, we use the trained model to predict segmentation masks for the test images. We calculate the Intersection over Union (IoU), Dice coefficient, and accuracy for each image, and save the results as images.


In [10]:
output_dir = '/kaggle/working/results/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

total_images = len(test_images)
current_image = 0

for i in range(len(test_images)):
    current_image += 1
    
    # Predict using U-Net
    pred_mask = unet_model.predict(np.expand_dims(test_images[i], axis=0))
    pred_mask = (pred_mask > 0.5).astype(np.uint8)[0, :, :, 0]
    
    # Calculate Metrics
    gt_mask = (test_masks[i] > 0).astype(np.uint8)
    intersection = np.sum((pred_mask == 1) & (gt_mask == 1))
    union = np.sum((pred_mask == 1) | (gt_mask == 1))
    
    iou = intersection / union
    dice = (2 * intersection) / (np.sum(pred_mask == 1) + np.sum(gt_mask == 1))
    accuracy = np.sum(pred_mask == gt_mask) / gt_mask.size

    # ✅ Now correctly plot the images
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    ax[0].imshow(test_images[i])
    ax[0].set_title("Original Image")
    
    ax[1].imshow(gt_mask, cmap='gray')
    ax[1].set_title("Ground Truth")
    
    ax[2].imshow(pred_mask, cmap='gray')
    ax[2].set_title(f"IoU: {iou:.4f}, Dice: {dice:.4f}, Acc: {accuracy:.4f}")
    
    # ✅ Save the image with metrics
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'image_{i+1}.png'))
    plt.close()
    
    # ✅ Show progress in terminal
    print(f"✅ Processed {current_image}/{total_images} | IoU: {iou:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
✅ Processed 1/110 | IoU: 0.1539
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
✅ Processed 2/110 | IoU: 0.4919
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
✅ Processed 3/110 | IoU: 0.7342
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
✅ Processed 4/110 | IoU: 0.8532
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
✅ Processed 5/110 | IoU: 0.5309
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
✅ Processed 6/110 | IoU: 0.3002
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
✅ Processed 7/110 | IoU: 0.6360
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
✅ Processed 8/110 | IoU: 0.0018
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
✅ Processed 9/110 | IoU: 0.2882
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
✅ Processed 10/110 | IoU: 0.0929
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
✅ Processed 11/110 | IoU: 0.0001
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
✅ Processed 12/110 | IoU: 0.4244
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
✅ Processed 13/110 | IoU: 0.1596
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
✅ Processed 14/110 | IoU: 0.6892
1/1 ━━━━━━━━━━━━━

### Step 6: Final Progress Message
After processing all images, we will print a message indicating that all images have been processed successfully.


In [11]:
shutil.make_archive('/kaggle/working/results', 'zip', '/kaggle/working/results')
print("✅ All 110 test images processed. Download results.zip")
from IPython.display import FileLink
FileLink('/kaggle/working/results.zip')

✅ All 110 test images processed. Download results.zip


/kaggle/working/results.zip